In [1]:
########## Import Libararies
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from tqdm.notebook import tqdm, trange
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import copy 
import pickle

import open3d as o3d
import trimesh
from compas.geometry import trimesh_remesh
from compas.datastructures import Mesh



%matplotlib inline

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
files = glob.glob(rf"D:\Data-to-KAIST\masks_npy\synthseg_wave1\*.npy")
areas = [] # 16 areas: Thalamus, Caudate, Putamen, Pallidum, Hippocampus, Amygdala, Accumbens, VentralDC 
classes = [49,50,51,52,53,54,58,60]#[10,11,12,13,17,18,26,28]
sub_num = 100
for i in range(len(classes)):
	areas.append(np.zeros((sub_num, 256, 163, 216)).astype(int))
for index in range(sub_num):
    mask = np.load(files[index])[:, 20:-20, 20:-20]
    for i, c in enumerate(classes):
        areas[i][index] = (mask == c).astype(int)


In [3]:
import pickle


# for i in range(len(classes)):
# 	where = np.where(areas[i] == 1)
# 	areas[i] = areas[i][:, where[1].min()-10:where[1].max()+10, where[2].min()-10:where[2].max()+10, where[3].min()-10:where[3].max()+10]
# 	print(areas[i].shape)

# with open('areas_right.pkl', 'wb') as f:
# 	pickle.dump(areas, f)
with open('areas.pkl', 'rb') as f:
	areas = pickle.load(f) 
for i in range(len(classes)):
	print(areas[i].shape)

(100, 56, 71, 98)
(100, 53, 97, 103)
(100, 56, 80, 99)
(100, 49, 60, 88)
(100, 60, 84, 89)
(100, 55, 60, 88)
(100, 47, 55, 91)
(100, 60, 68, 95)
(100, 56, 71, 98)
(100, 53, 97, 103)
(100, 56, 80, 99)
(100, 49, 60, 88)
(100, 60, 84, 89)
(100, 55, 60, 88)
(100, 47, 55, 91)
(100, 60, 68, 95)


In [4]:
########## PCA util functions
            
def marching_cubes_3d(data):
  vertices, triangles, _, _ = measure.marching_cubes(data, level=0)
  return vertices, triangles

def mesh_to_np(mesh, shape):
	# Generate queries for LV position
	shape = (shape[0], shape[1], shape[2], 3)
	# Create the array
	queries = np.zeros(shape, dtype=np.float32)
	# Fill the array with values
	for i in range(shape[0]):
		for j in range(shape[1]):
			for k in range(shape[2]):
				queries[i, j, k, :] = [i, j, k]
    
  ############## mesh to numpy occupancy ##############
	mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

  # Create a scene and add the triangle mesh
	scene = o3d.t.geometry.RaycastingScene()
	_ = scene.add_triangles(mesh)  # we do not need the geometry ID for mesh

	occupancy = scene.compute_occupancy(queries)
	return occupancy.numpy()

def generate_uniform_samples(mesh, num_samples):
  # Convert Open3D TriangleM esh to NumPy arrays
  
  vertices = np.asarray(mesh.vertices)
  triangles = np.asarray(mesh.triangles)
  mesh.compute_triangle_normals()
  tri_normals = np.asarray(mesh.triangle_normals)
  
  # Compute face areas
  face_areas = np.linalg.norm(np.cross(vertices[triangles[:, 0]] - vertices[triangles[:, 1]],
                                        vertices[triangles[:, 0]] - vertices[triangles[:, 2]]), axis=1) / 2.0

  # Normalize face areas to get probabilities for sampling
  probabilities = face_areas / np.sum(face_areas)

  # Sample faces based on the probabilities
  sampled_faces = np.random.choice(len(triangles), size=num_samples, p=probabilities, replace=True)

  # Sample points uniformly on the selected faces
  sampled_points = []
  for face_idx in sampled_faces:
    face = triangles[face_idx]
    barycentric_coords = np.random.rand(2, 1)
    barycentric_coords /= np.sum(barycentric_coords, axis=0)
    sampled_point = (1 - barycentric_coords[0] - barycentric_coords[1]) * vertices[face[0]] + \
                      barycentric_coords[0] * vertices[face[1]] + barycentric_coords[1] * vertices[face[2]]
    sampled_points.append(sampled_point)

  sampled_points = np.vstack(sampled_points)

  # Compute normals at the sampled points
  sampled_normals= tri_normals[sampled_faces]
  # print(sampled_normals.shape, sampled_points.shape)
  
  return sampled_points, sampled_normals

In [6]:
# Create Mean Shape

#LVs = np.load('../data/LV_left.npy')
all_meshes = []
all_voxels = []
all_pcds = []
all_reg_p2ps = []
all_visual_list = []
all_visual_pcd_list = []

for class_label in range(len(classes)):
	pt_num = 1000
	lv_num = areas[class_label].shape[0]
	voxel_size = areas[class_label].shape[1:]

	# Save 3D representations
	meshes = []
	voxels = []
	pcds = []
	reg_p2ps = []
	visual_list = []
	visual_pcd_list = []

	# Save meshes, pcd
	for i in range(lv_num):
		
		vertices, triangles = marching_cubes_3d(areas[class_label][i, :,:,:])
		
		mesh = o3d.geometry.TriangleMesh()
		mesh.vertices = o3d.utility.Vector3dVector(vertices)
		mesh.triangles = o3d.utility.Vector3iVector(triangles)
		mesh=o3d.geometry.TriangleMesh.filter_smooth_laplacian(mesh, number_of_iterations=3)

		points, normals = generate_uniform_samples(mesh, pt_num)
		pcd = o3d.geometry.PointCloud(points = o3d.utility.Vector3dVector(points))
		pcd.normals = o3d.utility.Vector3dVector(normals)
		meshes.append(mesh)
		pcds.append(pcd)

		# Register meshes to mesh0
		if i!=0:
			reg_p2p = o3d.pipelines.registration.registration_icp(
					pcds[i], pcds[0], max_correspondence_distance=20,  # Adjust the distance threshold as needed
					estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
					criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))
			meshes[i].transform(reg_p2p.transformation)
			pcds[i].transform(reg_p2p.transformation)

			voxels.append(mesh_to_np(meshes[i], shape=areas[class_label][0].shape))
			print(areas[class_label][0].shape, voxels[-1].shape)
			reg_p2ps.append(reg_p2p)
		meshes[i].paint_uniform_color([i*2/255,1-i*2/255,0.6])
		visual_list.append(meshes[i])
		visual_pcd_list.append(pcds[i])
	all_meshes.append(meshes)
	all_voxels.append(voxels)
	all_pcds.append(pcds)
	all_reg_p2ps.append(reg_p2ps)
	all_visual_list.append(visual_list)
	all_visual_pcd_list.append(visual_pcd_list)
  
	# o3d.visualization.draw_geometries(visual_list, mesh_show_wireframe=False, point_show_normal= False)

(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98) (56, 71, 98)
(56, 71, 98)

In [ ]:
import numpy as np
from scipy import ndimage

o3d.visualization.draw_geometries([visual_list[10]], mesh_show_wireframe=False, point_show_normal= False)
# with open("all_voxels_right.pkl", "wb") as f:
# 	pickle.dump(all_voxels, f)
print(len(all_voxels[0]), (all_voxels[0][3].shape))
for i in range (len(classes)):
  print(len(all_voxels[i]), (all_voxels[i][3].shape))
# Erosion and Dilation
def apply_morphology(grid, structure_size=3):
    # Define a structuring element (cube)
    struct_elem = np.ones((structure_size, structure_size, structure_size))
    
    # Apply dilation followed by erosion (Closing operation)
    dilated_grid = ndimage.binary_dilation(grid, structure=struct_elem)
    eroded_grid = ndimage.binary_erosion(dilated_grid, structure=struct_elem)
    
    return eroded_grid

99 (56, 71, 98)
99 (56, 71, 98)
99 (53, 97, 103)
99 (56, 80, 99)
99 (49, 60, 88)
99 (60, 84, 89)
99 (55, 60, 88)
99 (47, 55, 91)
99 (60, 68, 95)


In [10]:
with open("all_voxels_right.pkl", "rb") as f:
	all_voxels= pickle.load(f)
 
temp_list = []
for i in range (len(classes)):
  # Mean shape
	voxels= all_voxels[i].copy()
	voxels = np.stack(voxels, axis=0)
	print(voxels.shape)
	LV_sum= np.sum(voxels, axis = 0)/voxels.shape[0]
	LV_sum[LV_sum<0.3]=0
	LV_sum = apply_morphology(LV_sum, structure_size=3)

	vertices, triangles = marching_cubes_3d(LV_sum)
	mesh_avg_com = o3d.geometry.TriangleMesh()
	mesh_avg_com.vertices = o3d.utility.Vector3dVector(vertices)
	mesh_avg_com.triangles = o3d.utility.Vector3iVector(triangles)
	mesh_avg_com=o3d.geometry.TriangleMesh.filter_smooth_taubin(mesh_avg_com, number_of_iterations=2)

	mesh_tri = Mesh.from_vertices_and_faces(vertices=np.asarray(mesh_avg_com.vertices).tolist(),
												faces=np.asarray(mesh_avg_com.triangles).tolist())                       
	vert, fac = trimesh_remesh(mesh_tri.to_vertices_and_faces(), target_edge_length=1.5)
	print(len(vert))
	mesh_avg= o3d.geometry.TriangleMesh()
	mesh_avg.vertices = o3d.utility.Vector3dVector(np.asarray(vert))
	mesh_avg.triangles = o3d.utility.Vector3iVector(np.asarray(fac))
	mesh_avg=o3d.geometry.TriangleMesh.filter_smooth_simple(mesh_avg, number_of_iterations=2)

	mesh_tri = Mesh.from_vertices_and_faces(vertices=np.asarray(mesh_avg.vertices).tolist(),
												faces=np.asarray(mesh_avg.triangles).tolist())      
	vert, fac = trimesh_remesh(mesh_tri.to_vertices_and_faces(), target_edge_length=1.5)
	print(len(vert))
	mesh_avg= o3d.geometry.TriangleMesh()
	mesh_avg.vertices = o3d.utility.Vector3dVector(np.asarray(vert))
	mesh_avg.triangles = o3d.utility.Vector3iVector(np.asarray(fac))

	tri = np.asarray(mesh_avg.triangles)
	tri0 = tri.copy()
	tri[:,1]=tri0[:,2]
	tri[:,2]=tri0[:,1]
	mesh_avg.triangles = o3d.utility.Vector3iVector(tri)
	mesh_avg.paint_uniform_color([i*20/255,1-i*20/255,0.6])
	temp_list.append(mesh_avg)
 
	o3d.visualization.draw_geometries([mesh_avg], mesh_show_wireframe=True, point_show_normal=True)

(99, 56, 71, 98)
1472
1412
(99, 53, 97, 103)
1604
1492
(99, 56, 80, 99)
1601
1521
(99, 49, 60, 88)
642
600
(99, 60, 84, 89)
1572
1492
(99, 55, 60, 88)
664
631
(99, 47, 55, 91)
382
325
(99, 60, 68, 95)
1391
1303


In [11]:
# move to the center of mass and save as obj files
coord_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20)
for i in range(len(classes)):
  mesh = temp_list[i]
  vert = np.asarray(mesh.vertices)
  print(np.mean(vert, axis=0))
  vert = vert - np.mean(vert, axis=0)
  mesh.vertices = o3d.utility.Vector3dVector(vert)
  o3d.visualization.draw_geometries([mesh, coord_mesh], mesh_show_wireframe=True, point_show_normal=True)
  o3d.io.write_triangle_mesh(f"./{classes[i]}_temp.obj", mesh)

[33.06088492 33.44305048 56.897328  ]
[29.66794336 52.847593   55.96399813]
[31.06206544 40.72185771 55.08049714]
[28.85888481 30.43807898 49.74789284]
[34.39764484 42.22287934 48.05333193]
[33.58982208 31.66380967 49.37429483]
[29.61910688 27.02057832 49.89533742]
[37.45481494 33.7991067  52.66812664]


In [59]:
import copy
for i in range(len(classes)):
  mesh = o3d.io.read_triangle_mesh(f"./temp_meshes/{classes[i]}_L_temp.obj")
  r_mesh = copy.deepcopy(mesh)
  vert = np.asarray(r_mesh.vertices)
  vert[:,0] = -vert[:,0]
  tri = np.asarray(r_mesh.triangles)
  tri0 = tri.copy()
  tri[:,0]=tri0[:,2]
  tri[:,2]=tri0[:,0]
  
  r_mesh.paint_uniform_color([1,0,0])
  # o3d.visualization.draw_geometries([r_mesh, mesh], mesh_show_wireframe=True, point_show_normal=True)
  o3d.io.write_triangle_mesh(f"./temp_meshes/{classes[i]}_R_temp.obj", mesh)